In [1]:
import ast
import bs4
import time
import json
import requests
import pandas as pd
import scipy as scipy
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
import matplotlib.pyplot as plt

%matplotlib inline

In [13]:
df = pd.read_csv('canna_df.csv')
df['Feelings'] = df['Feelings'].map(ast.literal_eval) ## To get dicts instead of strings
df['Cannabinoids'] = df['Cannabinoids'].map(ast.literal_eval)

In [14]:
## Open dictionaries in columns ['Feelings','Cannabinoids'] to columns


df = df.reset_index(drop=True) 


for i in range(5): 
    df[f'Feeling_{i+1}'] = df.apply(lambda x: x['Feelings']['Feelings'][i],axis=1) ## Turn feelings list to columns
    df[f'Negative_{i+1}'] = df.apply(lambda x: x['Feelings']['Negatives'][i],axis=1)   
    df[f'Helps with_{i+1}'] = df.apply(lambda x: x['Feelings']['Helps with'][i],axis=1) 

    df[f'Feeling_{i+1}'] = df[f'Feeling_{i+1}'].str.replace(r'\b $','',regex=True) ## Remove space at the end of values
    df[f'Negative_{i+1}'] = df[f'Negative_{i+1}'].str.replace(r'\b $','',regex=True)
    df[f'Helps with_{i+1}'] = df[f'Helps with_{i+1}'].str.replace(r'\b $','',regex=True)


df['THC'] = df.apply(lambda x: x['Cannabinoids']['THC'],axis=1) ## Turn Cannabinoid dict to columns
df['CBD'] = df.apply(lambda x: x['Cannabinoids']['CBD'],axis=1) 
df['CBG'] = df.apply(lambda x: x['Cannabinoids']['CBG'],axis=1) 


df.drop(['Feelings','Cannabinoids'],inplace=True,axis=1) ## Remove old columns

cols = ['Rating','Rating Users','THC','CBD','CBG']

df['THC'] = df['THC'].str.replace("%",'',regex=True) # Remove % from numbers
df['CBD'] = df['CBD'].str.replace("%",'',regex=True)
df['CBG'] = df['CBG'].str.replace("%",'',regex=True)


df['CBD'].replace("—",np.nan,regex=True,inplace=True) # Replace '-' values with NaN
df['THC'].replace("—",np.nan,regex=True,inplace=True)
df['CBG'].replace("—",np.nan,regex=True,inplace=True)

df[cols] = df[cols].apply(pd.to_numeric) # Change columns type to numeric




In [15]:
def get_nan_replace_value(df,parent,mode,col):

    func_df = df.loc[(df['Left Parent'] == parent) | (df['Right Parent'] == parent) | (df['Strain Name'] == parent)] ## All rows with same parent, including the parent

    if mode == 'Feelings' or mode == 'Terpenes':
        try:
            feeling = func_df[col].value_counts().idxmax() ## Limits to look only at the same col

        except Exception as e:
            feeling = np.NaN

        return feeling

    elif mode == 'Cannabinoids':

        mean = round(func_df[col].mean(),0)
        
        return mean



def complete_nan(df,col_names):
        
    for i,col in enumerate(col_names,0):
        print(f'Working on {col}')
        if 'Feeling' in col: ## Create DF that contains 'Feeling' columns with NaN that has at least 1 parent that's not NaN
            nanim = df[(df['Feeling_1'].isnull()) & (df['Feeling_2'].isnull()) & (df['Feeling_3'].isnull()) & (df['Feeling_4'].isnull()) & (df['Feeling_5'].isnull()) & ((df['Left Parent'].notna()) | (df['Right Parent'].notna()))].copy()
            mode = 'Feelings'

        elif 'Negative' in col:
            nanim = df[(df['Negative_1'].isnull()) & (df['Negative_2'].isnull()) & (df['Negative_3'].isnull()) & (df['Negative_4'].isnull()) & (df['Negative_5'].isnull()) & ((df['Left Parent'].notna()) | (df['Right Parent'].notna()))].copy()
            mode = 'Feelings'
        
        elif 'Helps with' in col: ## Might be better to put in for loop
            nanim = df[(df['Helps with_1'].isnull()) & (df['Helps with_2'].isnull()) & (df['Helps with_3'].isnull()) & (df['Helps with_4'].isnull()) & (df['Helps with_5'].isnull()) & ((df['Left Parent'].notna()) | (df['Right Parent'].notna()))].copy()
            mode = 'Feelings'
        
        else: ## Create DF that contains a 'Cannabinoid' column with NaN, with at least 1 parent that's not NaN
            nanim = df[(df[col].isna()) & ((df['Left Parent'].notna()) | (df['Right Parent'].notna()))].copy() # To apply changes on real DF
            if col == 'Top Terpene':
                mode = 'Terpenes'
            else:
                mode = 'Cannabinoids'
        
        try:
            nanim[col] = nanim.apply(lambda x: get_nan_replace_value(df,x['Left Parent'],mode,col),axis=1)
            nanim[col] = nanim.apply(lambda x: get_nan_replace_value(df,x['Right Parent'],mode,col) if pd.isnull(x[col]) else x[col],axis=1)
            
            df[col].fillna(nanim[col],inplace=True)


        except Exception as e:
            print(e)
            pass
    

    return df

In [ ]:
df['Top Terpene'].isna().sum()

In [16]:
cols = ['Feeling_1','Feeling_2','Feeling_3','Feeling_4','Feeling_5','Negative_1','Negative_2','Negative_3','Negative_4','Negative_5','Helps with_1','Helps with_2','Helps with_3','Helps with_4','Helps with_5','THC','CBD','CBG','Top Terpene']
feelings_complete = complete_nan(df,cols) ## Fill NaN

Working on Feeling_1
Working on Feeling_2
Working on Feeling_3
Working on Feeling_4
Working on Feeling_5
Working on Negative_1
Working on Negative_2
Working on Negative_3
Working on Negative_4
Working on Negative_5
Working on Helps with_1
Working on Helps with_2
Working on Helps with_3
Working on Helps with_4
Working on Helps with_5
Working on THC
Working on CBD
Working on CBG
Working on Top Terpene


In [ ]:
# feelings_complete.head()
df['Top Terpene'].isna().sum()

In [17]:
df = df.copy()
df.update(feelings_complete)
df.columns.str.match("Unnamed") ## A new column was created from some reason
df = df.loc[:,~df.columns.str.match("Unnamed")]

In [ ]:
# df['Top Terpene'].isna().sum()
df.head(10)

In [18]:


without_rating_and_type = df[(df['Rating Users'].notna()) & (df['Type'].notna()) & (df['Strain Name'].notna())].copy()
# without_type = without_rating[without_rating['Type'].notna()].copy()


In [19]:
without_rating_and_type['CBD'].isna().sum()

2339

In [20]:
# without_rating.head(10)

# without_rating_and_type.shape

# without_rating_and_type[without_rating_and_type['THC'].isna()]

without_rating_and_type.dropna(subset=['THC'],inplace=True)

without_rating_and_type.head(10)


# curr = without_type[(without_type['Left Parent'].isna()) & (without_type['Right Parent'].isna()) & (without_type['THC'].isna())].copy()
# curr[curr['Rating Users'] < 10]

,Strain Name,Type,Rating,Rating Users,Left Parent,Right Parent,Left Child,Right Child,Top Terpene,Feeling_1,...,Helps with_3,Feeling_4,Negative_4,Helps with_4,Feeling_5,Negative_5,Helps with_5,THC,CBD,CBG
2,Lucid Dream,Hybrid,4.6,108.0,Blue Dream,Amnesia Haze,NaN,NaN,Myrcene,Uplifted,...,Stress,Euphoric,Anxious,Pain,Creative,Headache,Fatigue,20.0,11.0,1.0
3,Fred Flipn’ Stoned,Indica,5.0,2.0,Pink Champagne,Straight A's Haze,NaN,NaN,Myrcene,Relaxed,...,ADD,Tingly,NaN,PMS,Focused,NaN,NaN,18.0,NaN,1.0
4,Black Magic,Indica,4.5,17.0,NaN,NaN,NaN,NaN,Myrcene,Relaxed,...,Fatigue,Sleepy,NaN,Insomnia,Focused,NaN,Lack of appetite,18.0,NaN,NaN
6,NYC Diesel,Hybrid,4.2,939.0,Afghani,Mexican,Blue Diesel,Strawberry Diesel,Myrcene,Happy,...,Depression,Relaxed,Dizzy,Pain,Energetic,Headache,Nausea,18.0,4.0,1.0
7,Purple Goat,Hybrid,4.2,5.0,Trainwreck,Blueberry Skunk,NaN,NaN,Myrcene,Happy,...,Insomnia,Talkative,NaN,Pain,Creative,NaN,Fatigue,15.0,1.0,1.0
8,Alphadawg,Hybrid,5.0,1.0,Chemdawg,NaN,NaN,NaN,Myrcene,Relaxed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,1.0,1.0
11,Sour Sunset,Hybrid,4.7,49.0,Sour Diesel,NaN,NaN,NaN,Myrcene,Relaxed,...,Anxiety,Uplifted,Dizzy,Headaches,Focused,Headache,Pain,18.0,6.0,1.0
12,Purple Chemdawg,Indica,4.5,170.0,Granddaddy Purple,Chemdawg,NaN,NaN,Caryophyllene,Relaxed,...,Anxiety,Uplifted,Headache,Depression,Sleepy,Anxious,Insomnia,17.0,7.0,1.0
13,Vortex,Sativa,4.3,272.0,Space Queen,Apollo 13,Cinex,Timewreck,Myrcene,Euphoric,...,Depression,Energetic,Headache,Pain,Creative,Paranoid,Fatigue,16.0,NaN,1.0
15,Gutbuster,Indica,4.7,17.0,Cookies and Cream,Kimbo Kush,NaN,NaN,Limonene,Relaxed,...,Insomnia,Giggly,Paranoid,Pain,Creative,NaN,Anxiety,18.0,14.0,1.0


In [21]:
without_rating_and_type = without_rating_and_type.drop(columns=['Left Parent','Right Parent','Left Child','Right Child','CBG'])

In [24]:
# without_rating_and_type.head(10)

# without_rating_and_type['Top Terpene'].isna().sum()
# without_rating_and_type.head(10)

# df = without_rating_and_type.copy()
df.shape

(3302, 22)

In [28]:
df = df[(df['Feeling_1'].notna()) & (df['Feeling_2'].notna()) & (df['Negative_2'].notna()) & (df['Negative_1'].notna()) & (df['Helps with_2'].notna()) & (df['Helps with_1'].notna())].copy()

In [29]:
## Fill rest of NaNs with 'N/A' or 0.0

for i in range(1,6):
    df[f'Feeling_{i}'].fillna('N/A',inplace=True)
    df[f'Negative_{i}'].fillna('N/A',inplace=True)
    df[f'Helps with_{i}'].fillna('N/A',inplace=True)

df.dropna(subset=['THC'],inplace=True)
df['CBD'].fillna(0.0,inplace=True)
df['Top Terpene'].fillna('N/A',inplace=True)


In [11]:
without_rating_and_type.head(10)

,Strain Name,Type,Rating,Rating Users,Top Terpene,Feeling_1,Negative_1,Helps with_1,Feeling_2,Negative_2,...,Negative_3,Helps with_3,Feeling_4,Negative_4,Helps with_4,Feeling_5,Negative_5,Helps with_5,THC,CBD
2,Lucid Dream,Hybrid,4.6,108.0,Myrcene,Uplifted,Dry mouth,Depression,Happy,Dry eyes,...,Paranoid,Stress,Euphoric,Anxious,Pain,Creative,Headache,Fatigue,20.0,11.0
3,Fred Flipn’ Stoned,Indica,5.0,2.0,Myrcene,Relaxed,Dry mouth,Stress,Happy,N/A,...,N/A,ADD,Tingly,N/A,PMS,Focused,N/A,N/A,18.0,0.0
4,Black Magic,Indica,4.5,17.0,Myrcene,Relaxed,Dry mouth,Anxiety,Happy,Dry eyes,...,N/A,Fatigue,Sleepy,N/A,Insomnia,Focused,N/A,Lack of appetite,18.0,0.0
6,NYC Diesel,Hybrid,4.2,939.0,Myrcene,Happy,Dry mouth,Stress,Uplifted,Dry eyes,...,Paranoid,Depression,Relaxed,Dizzy,Pain,Energetic,Headache,Nausea,18.0,4.0
7,Purple Goat,Hybrid,4.2,5.0,Myrcene,Happy,Dry mouth,Stress,Uplifted,N/A,...,N/A,Insomnia,Talkative,N/A,Pain,Creative,N/A,Fatigue,15.0,1.0
8,Alphadawg,Hybrid,5.0,1.0,Myrcene,Relaxed,Dry mouth,Stress,Happy,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,16.0,1.0
11,Sour Sunset,Hybrid,4.7,49.0,Myrcene,Relaxed,Dry mouth,Stress,Happy,Dry eyes,...,Anxious,Anxiety,Uplifted,Dizzy,Headaches,Focused,Headache,Pain,18.0,6.0
12,Purple Chemdawg,Indica,4.5,170.0,Caryophyllene,Relaxed,Dry mouth,Pain,Euphoric,Dry eyes,...,Dizzy,Anxiety,Uplifted,Headache,Depression,Sleepy,Anxious,Insomnia,17.0,7.0
13,Vortex,Sativa,4.3,272.0,Myrcene,Euphoric,Dry mouth,Stress,Happy,Dry eyes,...,Dizzy,Depression,Energetic,Headache,Pain,Creative,Paranoid,Fatigue,16.0,0.0
15,Gutbuster,Indica,4.7,17.0,Limonene,Relaxed,Anxious,Stress,Euphoric,Dry eyes,...,Dry mouth,Insomnia,Giggly,Paranoid,Pain,Creative,N/A,Anxiety,18.0,14.0


In [30]:

df.to_csv('clean_df3.csv')